In [2]:
# trying to combine the data from the pyradiomics features and the VAE features
# load features from VAE and features from pyradiomics simultaneously


In [1]:
import os
import sys

sys.path.insert(1, "/labs/gevaertlab/users/yyhhli/code/vae/")

In [2]:

    from torch.utils.data.dataloader import DataLoader
    from evaluations.export import Exporter
    from datasets.utils import sitk2tensor
    from datasets import PATCH_DATASETS

    # stfrg_train_patch = PATCH_DATASETS["StanfordRadiogenomicsPatchAugDataset"](root_dir=None,
    #                                                                            transform=sitk2tensor,
    #                                                                            split='train')
    # stfrg_train_patch_dataloader = DataLoader(dataset=stfrg_train_patch,
    #                                           batch_size=1,
    #                                           shuffle=False,
    #                                           drop_last=False,
    #                                           num_workers=4,
    #                                           pin_memory=True)
    # stfrg_test_patch = PATCH_DATASETS["StanfordRadiogenomicsPatchDataset"](root_dir=None,
    #                                                                        transform=sitk2tensor,
    #                                                                        split='test')
    # stfrg_test_patch_dataloader = DataLoader(dataset=stfrg_test_patch,
    #                                          batch_size=1,
    #                                          shuffle=False,
    #                                          drop_last=False,
    #                                          num_workers=4,
    #                                          pin_memory=True)

stfrg_patch = PATCH_DATASETS["StanfordRadiogenomicsPatchAugDataset"](root_dir=None,
                                                                    transform=sitk2tensor,
                                                                    split='all')
stfrg_dl = DataLoader(dataset=stfrg_patch,
                      batch_size=1,
                                             shuffle=False,
                                             drop_last=False,
                                             num_workers=4,
                                             pin_memory=True)

/labs/gevaertlab/users/yyhhli/miniconda3/envs/lungvae38/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
tasks = ["StfAJCC", "StfEGFRMutation", "StfLymphInvasion", "StfNStage", "StfTStage"]

In [4]:
exporter = Exporter(base_model_name="VAE3D",
                                 log_name="VAE3D32AUG",
                                 version=70,
                                 task_names=tasks[0],
                                 dataloaders={"stf_all": stfrg_dl})
embeddings, data_names = exporter.get_embeddings()
labels = exporter.get_labels(
    label_name=tasks[0], label_kwds={}, data_names=data_names)

[06-29 21:02:37 | instantiator:  <module>] Created a temporary directory at /tmp/tmpm9hpgn2x
[06-29 21:02:37 | instantiator:    _write] Writing /tmp/tmpm9hpgn2x/_remote_module_non_sriptable.py
[06-29 21:02:40 | patch_ds:LIDCPatchAugDataset] patient split: train:699, val:88, test:88
[06-29 21:02:40 | patch_ds:LIDCPatchAugDataset] patch split: train, 33366
[06-29 21:02:40 | patch_ds:LNDbPatch32AugDataset] patient split: train:168, val:22, test:22
[06-29 21:02:40 | patch_ds:LNDbPatch32AugDataset] patch split: train, 4908
[06-29 21:02:40 | patch_stanfordradiogenomics:StanfordRadiogenomicsPatchAugDataset] patient split: train:100, test:43
[06-29 21:02:40 | export:  Exporter] initializing embeddings
[06-29 21:02:43 | patch_ds:LIDCPatchAugDataset] patient split: train:699, val:88, test:88
[06-29 21:02:44 | patch_ds:LIDCPatchAugDataset] patch split: train, 33366
[06-29 21:02:44 | patch_ds:LNDbPatch32AugDataset] patient split: train:168, val:22, test:22
[06-29 21:02:44 | patch_ds:LNDbPatch32Aug

 | 5.0 secs.


In [7]:
# stack X and Y
# import numpy as np
# embeddings = np.concatenate(
#         (embeddings['train'], embeddings['val']), axis=0)
# labels = np.concatenate(
#         (labels['train'], labels['val']), axis=0)

# if isinstance(embeddings['train'], pd.DataFrame):
#     embeddings = pd.concat(
#         [embeddings['train'], embeddings['val']], axis=0).reset_index()
# elif isinstance(embeddings['train'], np.ndarray):
#     embeddings = np.concatenate(
#         (embeddings['train'], embeddings['val']), axis=0)
# if isinstance(labels['train'], pd.DataFrame):
#     labels = pd.concat(
#         [labels['train'], labels['val']], axis=0).reset_index()
# elif isinstance(labels['train'], np.ndarray):
#     labels = np.concatenate(
#         (labels['train'], labels['val']), axis=0)

KeyError: 'train'

In [6]:
import numpy as np
np.array(embeddings['stf_all']).shape

(143, 4096)

In [8]:
import pandas as pd
import os.path as osp

# load the pyra features
feature_df = pd.read_csv(
    "/labs/gevaertlab/users/yyhhli/code/vae/baselines/pyradiomics/pyradiomics_features_default.csv")
numeric_cols = feature_df.select_dtypes(include=["int", "float"]).columns
Xdf = feature_df[numeric_cols]
X = Xdf.values

# get file names
image_paths = feature_df["Image"].values
file_names = [osp.basename(path).split("_")[0] for path in image_paths]

In [9]:
from applications.cv_models import cv_predict_task

/labs/gevaertlab/users/yyhhli/miniconda3/envs/lungvae38/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [10]:
np.concatenate([np.array(embeddings['stf_all']), X], axis=1).shape

(143, 4213)

In [11]:
not_na = (np.array(labels['stf_all']) != "NA")

In [13]:
from applications.cv_models import cv_predict_eval_with_model
results = cv_predict_eval_with_model(task_type="classification",
                            X=np.concatenate([np.array(embeddings['stf_all']), X], axis=1,)[not_na],
                            Y=np.array(labels['stf_all'])[not_na],
                            model_name="xgboost",
                            tune_hparams=True,
                            verbose=True,
                            seed=9001)

[06-29 21:07:27 | cv_models:cv_predict_eval_with_model] ======xgboost======
[06-29 21:07:27 | models:predict_with_model] ======xgboost======
/labs/gevaertlab/users/yyhhli/miniconda3/envs/lungvae38/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


train Counter({'I': 78, 'II+': 47}) test Counter({'I': 9, 'II+': 5})
Fitting 3 folds for each of 4 candidates, totalling 12 fits
[Pipeline] ............ (step 1 of 2) Processing scaler, total=   0.0s
[21:07:27] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[Pipeline] ......... (step 2 of 2) Processing predictor, total=   0.6s
[CV] END predictor__learning_rate=0.01, predictor__n_estimators=100, predictor__reg_alpha=1; total time=   0.7s
[Pipeline] ............ (step 1 of 2) Processing scaler, total=   0.0s
[21:07:28] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/labs/gevaertlab/users/yyhhli/miniconda3/envs/lungvae38/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[Pipeline] ......... (step 2 of 2) Processing predictor, total=   0.5s
[CV] END predictor__learning_rate=0.01, predictor__n_estimators=100, predictor__reg_alpha=1; total time=   0.5s
[Pipeline] ............ (step 1 of 2) Processing scaler, total=   0.0s
[21:07:28] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/labs/gevaertlab/users/yyhhli/miniconda3/envs/lungvae38/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

[Pipeline] ......... (step 2 of 2) Processing predictor, total=   0.2s
[CV] END predictor__learning_rate=0.1, predictor__n_estimators=100, predictor__reg_alpha=10; total time=   0.2s
[Pipeline] ............ (step 1 of 2) Processing scaler, total=   0.0s
[21:08:50] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/labs/gevaertlab/users/yyhhli/miniconda3/envs/lungvae38/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
[06-29 21:08:50 | models:grid_cv_model] best parameters: {'predictor__learning_rate': 0.1, 'predictor__n_estimators': 100, 'predictor__reg_alpha': 1}
[06-29 21:08:50 | models:predict_with_model] result for CV:{'split0_test_score': array([0.66666667, 0.5952381 , 0.71428571, 0.5952381 ]), 'split1_test_score': array([0.66666667, 0.64285714, 0.71428571, 0.64285714]), 'split2_test_score': array([0.54761905, 0.64285714, 0.61904762, 0.69047619]), 'mean_test_score': array([0.62698413, 0.62698413, 0.68253968,

[Pipeline] ......... (step 2 of 2) Processing predictor, total=   0.5s
[Pipeline] ............ (step 1 of 2) Processing scaler, total=   0.0s
[21:08:50] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/labs/gevaertlab/users/yyhhli/miniconda3/envs/lungvae38/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
[06-29 21:08:51 | models:predict_with_model] result for validation set:{'Accuracy': 0.7692307692307693, 'F1': 0.7631975867269986, 'Precision': 0.767094017094017, 'Recall': 0.7692307692307693, 'AUROC': 0.8}


[Pipeline] ......... (step 2 of 2) Processing predictor, total=   0.5s


In [15]:
len(results)

10

In [19]:
organized_results = [r[0] for r in results]
organized_results = {k: [dic[k] for dic in organized_results] for k in organized_results[0].keys()}

In [20]:
organized_results

{'Accuracy': [0.7142857142857143,
  0.9285714285714286,
  0.7142857142857143,
  0.5,
  0.5,
  0.6428571428571429,
  0.5714285714285714,
  0.6428571428571429,
  0.9285714285714286,
  0.7692307692307693],
 'F1': [0.645021645021645,
  0.926482873851295,
  0.6928571428571428,
  0.42857142857142855,
  0.5076923076923077,
  0.5918367346938777,
  0.5392857142857144,
  0.6115288220551378,
  0.9274255156608098,
  0.7631975867269986],
 'Precision': [0.8021978021978021,
  0.9357142857142857,
  0.7056277056277056,
  0.375,
  0.5625,
  0.6071428571428571,
  0.5281385281385281,
  0.6493506493506492,
  0.9365079365079365,
  0.767094017094017],
 'Recall': [0.7142857142857143,
  0.9285714285714286,
  0.7142857142857143,
  0.5,
  0.5,
  0.6428571428571429,
  0.5714285714285714,
  0.6428571428571429,
  0.9285714285714286,
  0.7692307692307693],
 'AUROC': [0.5333333333333333,
  0.9777777777777777,
  0.6888888888888889,
  0.4222222222222223,
  0.5555555555555555,
  0.5333333333333334,
  0.5777777777777778,

In [18]:
results[0][0]

{'Accuracy': 0.7142857142857143,
 'F1': 0.645021645021645,
 'Precision': 0.8021978021978021,
 'Recall': 0.7142857142857143,
 'AUROC': 0.5333333333333333}